In [3]:
import os
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn\\research'

In [4]:
## command should be in CustomerChurn directory
os.chdir('../')
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn'

In [5]:
from pathlib import Path
import joblib

model = joblib.load(Path('artifacts/model/model.joblib'))


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    model_path : Path
    test_data_path : Path
    all_params : dict
    target_column : str
    scores_path : Path
    

In [7]:
from ChurnPrediction.constants import *
from ChurnPrediction.utils.common import *


In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directory([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN
        evaluation_config = EvaluationConfig(
            model_path = config.model_path,
            test_data_path = config.test_data_path,
            all_params = params,
            target_column = list(schema.keys())[0],
            scores_path = config.scores_path
        )

        return evaluation_config



In [24]:
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ChurnPrediction.utils.common import *
import pandas as pd

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def evaluate_model(self, model, X_test, y_test):

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Calculate evaluation scores
        scores = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred)
        }

        return scores
    
    def evaluation(self):
        try:
            model = joblib.load(self.config.model_path)
            test_data = pd.read_csv(self.config.test_data_path)
        
            # Separate features and labels
            X_test = test_data.drop(columns=[self.config.target_column], axis = 1)
            y_test = test_data[self.config.target_column]

            scores = self.evaluate_model(model, X_test, y_test)

            # Save the scores to a JSON file
            save_json(Path(self.config.scores_path), scores)

        except Exception as e:
            raise e        

In [25]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    model_evaluator = Evaluation(evaluation_config)
    model_evaluator.evaluation()

except Exception as e:
    raise e

[2023-10-21 11:33:40,105: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-21 11:33:40,105: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 11:33:40,119: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-21 11:33:40,127: INFO: common: directory: artifacts created successfully]
[2023-10-21 11:33:40,167: INFO: common: json file: scores.json saved successfully]
